In [ ]:
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.layers import LSTM, Dense, Embedding
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Sample text data
text = """
The quick brown fox jumps over the lazy dog.
A quick brown fox jumps over the lazy dog.
The lazy dog is not so lazy anymore.

"""


In [ ]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index)

In [ ]:
# Create input sequences and labels
input_sequences = []
token_listin=[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    token_listin.append(token_list)
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
print(token_listin)

In [ ]:
# Pad sequences for a consistent input shape
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [ ]:
input_sequences

In [ ]:
# Create predictors and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
X

In [ ]:
y

In [ ]:
# Build an LSTM model
model = keras.Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
# Train the model
model.fit(X, y, epochs=100, verbose=1)

In [ ]:
# Function to predict the next word
def predict_next_word(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted)
    predicted_word = [word for word, index in tokenizer.word_index.items() if index == predicted_word_index][0]
    return predicted_word

In [ ]:
# Test the model
seed_text = "The quick brown"
next_word = predict_next_word(seed_text)
print(f"Input: {seed_text}")
print(f"Next word: {next_word}")